Simple example of eXirt's use

Run all bullets in the sequence below!



Generate files?


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
do_download = True

path_content = '/content'
path_fig = '/out_fig'
path_csv = '/out_csv'
path_model = '/out_model'
path_irt = '/out_irt' 
path_dataset = '/*'

!rm -r $path_content$path_fig
!rm -r $path_content$path_csv
!rm -r $path_content$path_model
!rm -r $path_content$path_irt


!mkdir $path_content$path_fig
!mkdir $path_content$path_csv
!mkdir $path_content$path_model
!mkdir $path_content$path_irt

def dirByDataset(datasetName):
  !mkdir $path_content$path_fig$datasetName
  !mkdir $path_content$path_csv$datasetName
  !mkdir $path_content$path_model$datasetName
  !mkdir $path_content$path_irt$datasetName

rm: cannot remove '/content/out_fig': No such file or directory
rm: cannot remove '/content/out_csv': No such file or directory
rm: cannot remove '/content/out_model': No such file or directory
rm: cannot remove '/content/out_irt': No such file or directory


## Install dependences

In [2]:
!pip install openml
!pip install catsim

import openml
import pandas as pd
import statistics
import numpy as np
import random
import copy

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from google.colab import files


!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_OtML.py
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_MLtIRT.py
!wget https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_analysis.py



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 KB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=f412cf20863ba35c30718e5ca2ae48fdd4cab0771f425459a5d7827a8a9b4e89
  Stored in directory: /root/.cache/pip/wheels/a2/de/68/bf3972de3ecb31e32bef59a7f4c75f0687a3674c476b347c14
Successfully built liac-arff
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-01-07 14:22:15--  https://raw.githubusercontent.com/josesousaribeiro/eXirt-XAI-Benchmark/main/decodIRT/decodIRT_OtML.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

## eXirt tool

In [3]:
############## GENERAL CONTROL #######################
proposal_1_pool = 1
proposal_2_loop = 2
exec_proposal = proposal_2_loop

# verbose
verbose = False
verbose_graph = False

download_files = True

# performance
exec_accuracy = 1
exec_auc = 2
exec_performance = exec_accuracy
normalize_performance = True

####### IRT ################
#execute theta ou trueScore
exec_theta = 0
exec_trueScore = 1
exec_base_irt_score = exec_trueScore

# calculo de  theta
theta_sum = 0
theta_min = 1
theta_mean = 2
exec_calc_theta = theta_mean


# irt algorithm ‘ternary’, ‘dichotomous’, ‘fibonacci’, ‘golden’, ‘brent’, ‘bounded’ and ‘golden2’
irt_method_fibonacci = '-metodo fibonacci'
irt_method_bounded = '-metodo bounded'
irt_method_ternary = '-metodo ternary' #long time execution
irt_method_dichotomous = '-metodo dichotomous' #long time execution
irt_method_golden = '-metodo golden'
irt_method_brent = '-metodo brent'
irt_method_golden2 = '-metodo golden2' #problem in execution
irt_method = irt_method_bounded

# controle de propriedades do IRT
irt_discriminate = True #False remove the property by IRTs calc.
irt_difficulty = True #False remove the property by IRTs calc.
irt_divine = True #False remove the property by IRTs calc.

############## PROPOSAL 1 POOL #######################
include_original_clf_data_pool = True
number_of_features_deletion = 2 # values 1 to ++

list_clf_pool = [] #empity

############## PROPOSAL 2 LOOP #######################

include_original_clf_data_loop = True
number_of_features_variation = 2 # values 1 to 4

# test or train
exec_test = 0
exec_train = 1
exec_in = exec_test

#loop of models variation
exec_permutation = 0 # random base
exec_noise = 1 # random base
exec_zeros = 2
exec_norm = 3 
exec_ordup = 4
exec_orddown = 5
exec_inver = 6
exec_binning = 7
exec_mult_neg = 8
exec_mean = 9
exec_std = 10
exec_zscore = 11
exec_variation_method = [exec_mult_neg, exec_binning]

#list of thetas
list_clf_loop = [] #empity

########## global declaration #############

# model = None
# X_train = None
# X_test = None
# y_test = None
# y_train = None

def z_score_serie(s):
  # copy the dataframe
  s_std = s.copy()
  s_std = (s_std - s_std.mean()) / s_std.std()
  return s_std

def auc_score(y, y_pred):
    
    fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
    
    return metrics.auc(fpr, tpr)

def powerSetLimited(s,l):

    def powerset(s):
      x = len(s)
      masks = [1 << i for i in range(x)]
      for i in range(1 << x):
          yield [ss for mask, ss in zip(masks, s) if i & mask]
    
    psl = []
    ps = list(powerset(s))
    for _,i in enumerate(ps):
      if len(i) <= l and len(i)>0:
        psl.append(i)
    return psl

def run_prepare(model,data_exec_x, data_exec_y, X_train, X_test, y_train, y_test):
  global path_dataset
  
  %rm $path_content$path_irt'/tabela_base_para_executar_irt.csv'
  %rm $path_content$path_irt'/tabela_base_para_executar_irt_accuracy.csv'


  #prediction
  original_outputs = model.predict(data_exec_x)


  #XAI-IRT
  # train, test, model and outputs

  #Loop of models
  df_loop_of_models = pd.DataFrame()
  df_loop_of_models_performance = pd.DataFrame(columns=['Metodo','Acuracia'])



  if exec_proposal == proposal_2_loop:
    if verbose:
      print('Original data')
      print(data_exec_x)

    number_of_instances = len(data_exec_x)

    if include_original_clf_data_loop:
      str_tmp = 'Clf original data'
      if exec_performance == exec_accuracy:
        result_accuracy = accuracy_score(y_true = original_outputs, y_pred = original_outputs, normalize=normalize_performance) #accuracy
      else:
        if exec_performance == exec_auc:
          result_accuracy = auc_score(original_outputs, original_outputs) #AUC

      result_pred = (original_outputs == data_exec_y.values) #binarization
      df_loop_of_models[str_tmp] = result_pred.astype(int)[:] #boolean to int
      #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_tmp,'Acuracia':result_accuracy},ignore_index=True)
      df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_tmp, result_accuracy] 
    
    #inset variation of ONE attribute in loop of models
    if number_of_features_variation >= 1:
      for _, variation in enumerate(exec_variation_method):
        for id,c in enumerate(data_exec_x.columns):
          #criando cópia do dado inicial
          data_exec_x_copy = data_exec_x.copy()
          if variation == exec_permutation:
            #trocando posições de cada instância do atributo da vez
            random_id = random.sample(range(0,number_of_instances), number_of_instances)
            data_exec_x_copy[c] = data_exec_x_copy[c].values[random_id]
            method = 'Permutation'
          else:
            if variation == exec_noise:
              #aplica ruido a cada instâcia do atributo da vez
              noise = np.random.normal(0, 1, number_of_instances)
              data_exec_x_copy[c] = data_exec_x_copy[c] + noise
              method = 'Noise'
            else: 
              if variation == exec_zeros:
                #aplica zeros a cada instância do atributo da vez
                zeros = np.zeros(number_of_instances)
                data_exec_x_copy[c] = zeros
                method = 'Zeros'
              else:
                if variation == exec_norm:
                  #normaliza os elementos
                  norm = np.linalg.norm(data_exec_x_copy[c])
                  normal_array = data_exec_x_copy[c]/norm
                  data_exec_x_copy[c] = normal_array
                  method = 'Normalization'
                else:
                  if variation == exec_ordup:
                    #ordena em ordem crescente os elementos
                    order_up = np.sort(data_exec_x_copy[c])
                    data_exec_x_copy[c] = order_up
                    method = 'Ordernation_Up'
                  else:
                    if variation == exec_orddown:
                      #ordena em ordem decrescente os elementos
                      order_down = -np.sort(-data_exec_x_copy[c])
                      data_exec_x_copy[c] = order_down
                      method = 'Ordernation_Down'
                    else:
                      if variation == exec_inver:
                        #inverte os elementos
                        transp = np.flipud(data_exec_x_copy[c])
                        data_exec_x_copy[c] = transp
                        method = 'Invertion'
                      else:
                        if variation == exec_binning:
                          #inverte os elementos
                          mean_arr = np.mean(data_exec_x_copy[c])
                          binn = np.digitize(data_exec_x_copy[c],bins=[mean_arr])
                          data_exec_x_copy[c] = binn
                          method = 'Binning'
                        else:
                          if variation == exec_mult_neg:
                            #multiplica por -1
                            data_exec_x_copy[c] = data_exec_x_copy[c] * -1
                            method = 'MultNeg'
                          else:
                            if variation == exec_mean:
                              #mean
                              inst =  len(data_exec_x_copy[c])
                              data_exec_x_copy[c] = [statistics.mean(data_exec_x_copy[c])]*inst
                              method = 'Mean'
                            else:
                              if variation == exec_std:
                                #std
                                inst =  len(data_exec_x_copy[c])
                                data_exec_x_copy[c] = [statistics.stdev(data_exec_x_copy[c])]*inst
                                method = 'Std'
                              else:
                                if variation == exec_zscore:
                                  #zscore
                                  data_exec_x_copy[c] = z_score_serie(data_exec_x_copy[c])
                                  method = 'Zscore'

          if verbose:
            print('')
            print(method,' of ',c)
            print(data_exec_x_copy)
          if verbose_graph:
            data_exec_x_copy.plot.kde(by=data_exec_x_copy.columns, alpha=0.5,title=str(method+' of '+c))

          result_pred = model.predict(data_exec_x_copy) #prediction

          if exec_performance == exec_accuracy:
            result_accuracy = accuracy_score(y_true = original_outputs, y_pred = result_pred, normalize=normalize_performance) #accuracy
          else:
            if exec_performance == exec_auc:
              result_accuracy = auc_score(original_outputs, result_pred) #AUC

          result_pred = (result_pred == original_outputs) #binarization
          str_model_name = 'Clf '+method+' "'+str(c)+'"'
          list_clf_loop.append(str_model_name)
          df_loop_of_models[str_model_name] = result_pred.astype(int)[:]
          #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_model_name,'Acuracia':result_accuracy},ignore_index=True,verify_integrity=True)
          df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_model_name, result_accuracy]

    #inset invertion of TWO attributes in loop of models
    if number_of_features_variation >= 2:
      for _,variation in enumerate(exec_variation_method):
        for id1,c1 in enumerate(data_exec_x.columns):
          for id2,c2 in enumerate(data_exec_x.columns):
            if id2 > id1:
              #criando cópia do dado inicial
              data_exec_x_copy = data_exec_x.copy()
              if variation == exec_permutation:
                #trocando posições de cada instância do atributo da vez
                random_id = random.sample(range(0,number_of_instances), number_of_instances)
                data_exec_x_copy[c1] = data_exec_x_copy[c1].values[random_id]
                random_id = random.sample(range(0,number_of_instances), number_of_instances)
                data_exec_x_copy[c2] = data_exec_x_copy[c2].values[random_id]
                method = 'Permutation'
              else:
                if variation == exec_noise:
                  #aplica ruido a cada instâcia do atributo da vez
                  noise = np.random.normal(0, 1, number_of_instances)
                  data_exec_x_copy[c1] = data_exec_x_copy[c1] + noise
                  noise = np.random.normal(0, 1, number_of_instances)
                  data_exec_x_copy[c2] = data_exec_x_copy[c2] + noise
                  method = 'Noise'
                else:
                  if variation == exec_zeros:
                    #aplica zeros a cada instância do atributo da vez
                    zeros = np.zeros(number_of_instances)
                    data_exec_x_copy[c1] = zeros
                    data_exec_x_copy[c2] = zeros
                    method = 'Zeros'
                  else:
                    if variation == exec_norm:
                      #normaliza os elementos
                      norm = np.linalg.norm(data_exec_x_copy[c1])
                      normal_array = data_exec_x_copy[c1]/norm
                      data_exec_x_copy[c1] = normal_array
                      norm = np.linalg.norm(data_exec_x_copy[c2])
                      normal_array = data_exec_x_copy[c2]/norm
                      data_exec_x_copy[c2] = normal_array
                      method = 'Normalization'
                    else:
                      if variation == exec_ordup:
                        #ordena em ordem crescente os elementos
                        order_up = np.sort(data_exec_x_copy[c1])
                        data_exec_x_copy[c1] = order_up
                        order_up = np.sort(data_exec_x_copy[c2])
                        data_exec_x_copy[c2] = order_up
                        method = 'Ordernation_Up'
                      else:
                        if variation == exec_orddown:
                          #ordena em ordem decrescente os elementos
                          order_down = -np.sort(-data_exec_x_copy[c1])
                          data_exec_x_copy[c1] = order_down
                          order_down = -np.sort(-data_exec_x_copy[c2])
                          data_exec_x_copy[c2] = order_down
                          method = 'Ordernation_Down'
                        else:
                          if variation == exec_inver:
                            #inverte os elementos
                            transp = np.flipud(data_exec_x_copy[c1])
                            data_exec_x_copy[c1] = transp
                            transp = np.flipud(data_exec_x_copy[c2])
                            data_exec_x_copy[c2] = transp
                            method = 'Invertion'
                          else:
                            if variation == exec_binning:
                              #inverte os elementos
                              mean_arr = np.mean(data_exec_x_copy[c1])
                              binn = np.digitize(data_exec_x_copy[c1],bins=[mean_arr])
                              data_exec_x_copy[c1] = binn
                              mean_arr = np.mean(data_exec_x_copy[c2])
                              binn = np.digitize(data_exec_x_copy[c2],bins=[mean_arr])
                              data_exec_x_copy[c2] = binn
                              method = 'Binning'
                            else:
                              if variation == exec_mult_neg:
                                #multiplica por -1
                                data_exec_x_copy[c1] = data_exec_x_copy[c1] * -1
                                data_exec_x_copy[c2] = data_exec_x_copy[c2] * -1
                                method = 'MultNeg'
                              else:
                                if variation == exec_mean:
                                  #mean
                                  inst =  len(data_exec_x_copy[c1])
                                  data_exec_x_copy[c1] = [statistics.mean(data_exec_x_copy[c1])]*inst
                                  data_exec_x_copy[c2] = [statistics.mean(data_exec_x_copy[c2])]*inst
                                  method = 'Mean'
                                else:
                                  if variation == exec_std:
                                    #std
                                    inst =  len(data_exec_x_copy[c1])
                                    data_exec_x_copy[c1] = [statistics.stdev(data_exec_x_copy[c1])]*inst
                                    data_exec_x_copy[c2] = [statistics.stdev(data_exec_x_copy[c2])]*inst
                                    method = 'Std'
                                  else:
                                    if variation == exec_zscore:
                                      #zscore
                                      data_exec_x_copy[c1] = z_score_serie(data_exec_x_copy[c1])
                                      data_exec_x_copy[c2] = z_score_serie(data_exec_x_copy[c2])
                                      method = 'Zscore'

              str_model_name = 'Clf '+method+' "'+str(c1)+'" and "'+str(c2)+'"'
              if verbose:
                print('')
                print(str_model_name)
                print(data_exec_x_copy)
              if verbose_graph:
                data_exec_x_copy.plot.kde(by=data_exec_x_copy.columns, alpha=0.5,title=str_model_name)

              #executando o modelo com o atributo da vez embaralhado
              result_pred = model.predict(data_exec_x_copy) #prediction

              if exec_performance == exec_accuracy:
                result_accuracy = accuracy_score(y_true = original_outputs, y_pred = result_pred, normalize=normalize_performance) #accuracy
              else:
                if exec_performance == exec_auc:
                  result_accuracy = auc_score(original_outputs, result_pred) #AUC

              result_pred = (result_pred == original_outputs) #binarization
              str_model_name = 'Clf '+method+' "'+str(c1)+'" and "'+str(c2)+'"'
              list_clf_loop.append(str_model_name)
              df_loop_of_models[str_model_name] = result_pred.astype(int)[:]
              #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_model_name,'Acuracia':result_accuracy},ignore_index=True, verify_integrity=True)
              df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_model_name, result_accuracy]
            else:
              pass

    #inset invertion of Three attributes in loop of models
    if number_of_features_variation >= 3:
      for _, variation in enumerate(exec_variation_method):
        for id1,c1 in enumerate(data_exec_x.columns):
          for id2,c2 in enumerate(data_exec_x.columns):
            for id3,c3 in enumerate(data_exec_x.columns):
              if id3 > id2 and id2 > id1:
                #criando cópia do dado inicial
                data_exec_x_copy = data_exec_x.copy()
                if variation == exec_permutation:
                  #trocando posições de cada instância do atributo da vez
                  random_id = random.sample(range(0,number_of_instances), number_of_instances)
                  data_exec_x_copy[c1] = data_exec_x_copy[c1].values[random_id]
                  random_id = random.sample(range(0,number_of_instances), number_of_instances)
                  data_exec_x_copy[c2] = data_exec_x_copy[c2].values[random_id]
                  random_id = random.sample(range(0,number_of_instances), number_of_instances)
                  data_exec_x_copy[c3] = data_exec_x_copy[c3].values[random_id]
                  method = 'Permutation'
                else:
                  if variation == exec_noise:
                    #aplica ruido a cada instâcia do atributo da vez
                    noise = np.random.normal(0, 1, number_of_instances)
                    data_exec_x_copy[c1] = data_exec_x_copy[c1] + noise
                    noise = np.random.normal(0, 1, number_of_instances)
                    data_exec_x_copy[c2] = data_exec_x_copy[c2] + noise
                    noise = np.random.normal(0, 1, number_of_instances)
                    data_exec_x_copy[c3] = data_exec_x_copy[c3] + noise
                    method = 'Noise'
                  else:
                    if variation == exec_zeros:
                      #aplica zeros a cada instância do atributo da vez
                      zeros = np.zeros(number_of_instances)
                      data_exec_x_copy[c1] = zeros
                      data_exec_x_copy[c2] = zeros
                      data_exec_x_copy[c3] = zeros
                      method = 'Zeros'
                    else:
                      if variation == exec_norm:
                        #normaliza os elementos
                        norm = np.linalg.norm(data_exec_x_copy[c1])
                        normal_array = data_exec_x_copy[c1]/norm
                        data_exec_x_copy[c1] = normal_array
                        norm = np.linalg.norm(data_exec_x_copy[c2])
                        normal_array = data_exec_x_copy[c2]/norm
                        data_exec_x_copy[c2] = normal_array
                        normal_array = data_exec_x_copy[c3]/norm
                        data_exec_x_copy[c3] = normal_array
                        method = 'Normalization'
                      else:
                        if variation == exec_ordup:
                          #ordena em ordem crescente os elementos
                          order_up = np.sort(data_exec_x_copy[c1])
                          data_exec_x_copy[c1] = order_up
                          order_up = np.sort(data_exec_x_copy[c2])
                          data_exec_x_copy[c2] = order_up
                          order_up = np.sort(data_exec_x_copy[c3])
                          data_exec_x_copy[c3] = order_up
                          method = 'Ordernation_Up'
                        else:
                          if variation == exec_orddown:
                            #ordena em ordem decrescente os elementos
                            order_down = -np.sort(-data_exec_x_copy[c1])
                            data_exec_x_copy[c1] = order_down
                            order_down = -np.sort(-data_exec_x_copy[c2])
                            data_exec_x_copy[c2] = order_down
                            order_down = -np.sort(-data_exec_x_copy[c3])
                            data_exec_x_copy[c3] = order_down
                            method = 'Ordernation_Down'
                          else:
                            if variation == exec_inver:
                              #inverte os elementos
                              transp = np.flipud(data_exec_x_copy[c1])
                              data_exec_x_copy[c1] = transp
                              transp = np.flipud(data_exec_x_copy[c2])
                              data_exec_x_copy[c2] = transp
                              transp = np.flipud(data_exec_x_copy[c3])
                              data_exec_x_copy[c3] = transp
                              method = 'Invertion'
                            else:
                              if variation == exec_binning:
                                #inverte os elementos
                                mean_arr = np.mean(data_exec_x_copy[c1])
                                binn = np.digitize(data_exec_x_copy[c1],bins=[mean_arr])
                                data_exec_x_copy[c1] = binn
                                mean_arr = np.mean(data_exec_x_copy[c2])
                                binn = np.digitize(data_exec_x_copy[c2],bins=[mean_arr])
                                data_exec_x_copy[c2] = binn
                                mean_arr = np.mean(data_exec_x_copy[c3])
                                binn = np.digitize(data_exec_x_copy[c3],bins=[mean_arr])
                                data_exec_x_copy[c3] = binn
                                method = 'Binning'
                              else:
                                if variation == exec_mult_neg:
                                  #multiplica por -1
                                  data_exec_x_copy[c1] = data_exec_x_copy[c1] * -1
                                  data_exec_x_copy[c2] = data_exec_x_copy[c2] * -1
                                  data_exec_x_copy[c3] = data_exec_x_copy[c3] * -1
                                  method = 'MultNeg'
                                else:
                                  if variation == exec_mean:
                                    #mean
                                    inst =  len(data_exec_x_copy[c1])
                                    data_exec_x_copy[c1] = [statistics.mean(data_exec_x_copy[c1])]*inst
                                    data_exec_x_copy[c2] = [statistics.mean(data_exec_x_copy[c2])]*inst
                                    data_exec_x_copy[c3] = [statistics.mean(data_exec_x_copy[c3])]*inst
                                    method = 'Mean'
                                  else:
                                    if variation == exec_std:
                                      #std
                                      inst =  len(data_exec_x_copy[c1])
                                      data_exec_x_copy[c1] = [statistics.stdev(data_exec_x_copy[c1])]*inst
                                      data_exec_x_copy[c2] = [statistics.stdev(data_exec_x_copy[c2])]*inst
                                      data_exec_x_copy[c3] = [statistics.stdev(data_exec_x_copy[c3])]*inst
                                      method = 'Std'
                                    else:
                                      if variation == exec_zscore:
                                        #zscore
                                        data_exec_x_copy[c1] = z_score_serie(data_exec_x_copy[c1])
                                        data_exec_x_copy[c2] = z_score_serie(data_exec_x_copy[c2])
                                        data_exec_x_copy[c3] = z_score_serie(data_exec_x_copy[c3])
                                        method = 'Zscore'

                  
                str_model_name = 'Clf '+method+' "'+str(c1)+'" and "'+str(c2)+'" and "'+str(c3)+'"'
                if verbose:
                  print('')
                  print(str_model_name)
                  print(data_exec_x_copy)
                if verbose_graph:
                  data_exec_x_copy.plot.kde(by=data_exec_x_copy.columns, alpha=0.5,title=str_model_name)

                
                result_pred = model.predict(data_exec_x_copy) #prediction

                if exec_performance == exec_accuracy:
                  result_accuracy = accuracy_score(y_true = original_outputs, y_pred = result_pred, normalize=normalize_performance) #accuracy
                else:
                  if exec_performance == exec_auc:
                    result_accuracy = auc_score(original_outputs, result_pred) #AUC

                result_pred = (result_pred == original_outputs) #binarization
                list_clf_loop.append(str_model_name)
                df_loop_of_models[str_model_name] = result_pred.astype(int)[:]
                #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_model_name,'Acuracia':result_accuracy},ignore_index=True, verify_integrity=True)
                df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_model_name, result_accuracy]
              else:
                pass

    #inset invertion of Four attributes in loop of models
    if number_of_features_variation == 4:
      for _, variation in enumerate(exec_variation_method):
        for id1,c1 in enumerate(data_exec_x.columns):
          for id2,c2 in enumerate(data_exec_x.columns):
            for id3,c3 in enumerate(data_exec_x.columns):
              for id4,c4 in enumerate(data_exec_x.columns):
                if id4 > id3 and id3 > id2 and id2 > id1:
                  #criando cópia do dado inicial
                  data_exec_x_copy = data_exec_x.copy()
                  if variation == exec_permutation:
                    #trocando posições de cada instância do atributo da vez
                    random_id = random.sample(range(0,number_of_instances), number_of_instances)
                    data_exec_x_copy[c1] = data_exec_x_copy[c1].values[random_id]
                    random_id = random.sample(range(0,number_of_instances), number_of_instances)
                    data_exec_x_copy[c2] = data_exec_x_copy[c2].values[random_id]
                    random_id = random.sample(range(0,number_of_instances), number_of_instances)
                    data_exec_x_copy[c3] = data_exec_x_copy[c3].values[random_id]
                    random_id = random.sample(range(0,number_of_instances), number_of_instances)
                    data_exec_x_copy[c4] = data_exec_x_copy[c4].values[random_id]
                    method = 'Permutation'
                  else:
                    if variation == exec_noise:
                      #aplica ruido a cada instâcia do atributo da vez
                      noise = np.random.normal(0, 1, number_of_instances)
                      data_exec_x_copy[c1] = data_exec_x_copy[c1] + noise
                      noise = np.random.normal(0, 1, number_of_instances)
                      data_exec_x_copy[c2] = data_exec_x_copy[c2] + noise
                      noise = np.random.normal(0, 1, number_of_instances)
                      data_exec_x_copy[c3] = data_exec_x_copy[c3] + noise
                      noise = np.random.normal(0, 1, number_of_instances)
                      data_exec_x_copy[c4] = data_exec_x_copy[c4] + noise
                      method = 'Noise'
                    else:
                      if variation == exec_zeros:
                        #aplica zeros a cada instância do atributo da vez
                        zeros = np.zeros(number_of_instances)
                        data_exec_x_copy[c1] = zeros
                        data_exec_x_copy[c2] = zeros
                        data_exec_x_copy[c3] = zeros
                        data_exec_x_copy[c4] = zeros
                        method = 'Zeros'
                      else:
                        if variation == exec_norm:
                          #normaliza os elementos
                          norm = np.linalg.norm(data_exec_x_copy[c1])
                          normal_array = data_exec_x_copy[c1]/norm
                          data_exec_x_copy[c1] = normal_array
                          norm = np.linalg.norm(data_exec_x_copy[c2])
                          normal_array = data_exec_x_copy[c2]/norm
                          data_exec_x_copy[c2] = normal_array
                          normal_array = data_exec_x_copy[c3]/norm
                          data_exec_x_copy[c3] = normal_array
                          normal_array = data_exec_x_copy[c4]/norm
                          data_exec_x_copy[c4] = normal_array
                          method = 'Normalization'
                        else:
                          if variation == exec_ordup:
                            #ordena em ordem crescente os elementos
                            order_up = np.sort(data_exec_x_copy[c1])
                            data_exec_x_copy[c1] = order_up
                            order_up = np.sort(data_exec_x_copy[c2])
                            data_exec_x_copy[c2] = order_up
                            order_up = np.sort(data_exec_x_copy[c3])
                            data_exec_x_copy[c3] = order_up
                            order_up = np.sort(data_exec_x_copy[c4])
                            data_exec_x_copy[c4] = order_up
                            method = 'Ordernation_Up'
                          else:
                            if variation == exec_orddown:
                              #ordena em ordem decrescente os elementos
                              order_down = -np.sort(-data_exec_x_copy[c1])
                              data_exec_x_copy[c1] = order_down
                              order_down = -np.sort(-data_exec_x_copy[c2])
                              data_exec_x_copy[c2] = order_down
                              order_down = -np.sort(-data_exec_x_copy[c3])
                              data_exec_x_copy[c3] = order_down
                              order_down = -np.sort(-data_exec_x_copy[c4])
                              data_exec_x_copy[c4] = order_down
                              method = 'Ordernation_Down'
                            else:
                              if variation == exec_inver:
                                #inverte os elementos
                                transp = np.flipud(data_exec_x_copy[c1])
                                data_exec_x_copy[c1] = transp
                                transp = np.flipud(data_exec_x_copy[c2])
                                data_exec_x_copy[c2] = transp
                                transp = np.flipud(data_exec_x_copy[c3])
                                data_exec_x_copy[c3] = transp
                                transp = np.flipud(data_exec_x_copy[c4])
                                data_exec_x_copy[c4] = transp
                                method = 'Invertion'
                              else:
                                if variation == exec_binning:
                                  #inverte os elementos
                                  mean_arr = np.mean(data_exec_x_copy[c1])
                                  binn = np.digitize(data_exec_x_copy[c1],bins=[mean_arr])
                                  data_exec_x_copy[c1] = binn
                                  mean_arr = np.mean(data_exec_x_copy[c2])
                                  binn = np.digitize(data_exec_x_copy[c2],bins=[mean_arr])
                                  data_exec_x_copy[c2] = binn
                                  mean_arr = np.mean(data_exec_x_copy[c3])
                                  binn = np.digitize(data_exec_x_copy[c3],bins=[mean_arr])
                                  data_exec_x_copy[c3] = binn
                                  mean_arr = np.mean(data_exec_x_copy[c4])
                                  binn = np.digitize(data_exec_x_copy[c4],bins=[mean_arr])
                                  data_exec_x_copy[c4] = binn
                                  method = 'Binning'
                                else:
                                  if variation == exec_mult_neg:
                                    #multiplica por -1
                                    data_exec_x_copy[c1] = data_exec_x_copy[c1] * -1
                                    data_exec_x_copy[c2] = data_exec_x_copy[c2] * -1
                                    data_exec_x_copy[c3] = data_exec_x_copy[c3] * -1
                                    data_exec_x_copy[c4] = data_exec_x_copy[c4] * -1
                                    method = 'MultNeg'
                                  else:
                                    if variation == exec_mean:
                                      #mean
                                      inst =  len(data_exec_x_copy[c1])
                                      data_exec_x_copy[c1] = [statistics.mean(data_exec_x_copy[c1])]*inst
                                      data_exec_x_copy[c2] = [statistics.mean(data_exec_x_copy[c2])]*inst
                                      data_exec_x_copy[c3] = [statistics.mean(data_exec_x_copy[c3])]*inst
                                      data_exec_x_copy[c4] = [statistics.mean(data_exec_x_copy[c4])]*inst
                                      method = 'Mean'
                                    else:
                                      if variation == exec_std:
                                        #std
                                        inst =  len(data_exec_x_copy[c1])
                                        data_exec_x_copy[c1] = [statistics.stdev(data_exec_x_copy[c1])]*inst
                                        data_exec_x_copy[c2] = [statistics.stdev(data_exec_x_copy[c2])]*inst
                                        data_exec_x_copy[c3] = [statistics.stdev(data_exec_x_copy[c3])]*inst
                                        data_exec_x_copy[c4] = [statistics.stdev(data_exec_x_copy[c4])]*inst
                                        method = 'Std'
                                      else:
                                        if variation == exec_zscore:
                                          #zscore
                                          data_exec_x_copy[c1] = z_score_serie(data_exec_x_copy[c1])
                                          data_exec_x_copy[c2] = z_score_serie(data_exec_x_copy[c2])
                                          data_exec_x_copy[c3] = z_score_serie(data_exec_x_copy[c3])
                                          data_exec_x_copy[c4] = z_score_serie(data_exec_x_copy[c4])
                                          method = 'Zscore'
                  
                  str_model_name = 'Clf '+method+' "'+str(c1)+'" and "'+str(c2)+'" and "'+str(c3)+'" and "'+str(c4)+'"'
                  if verbose:
                    print('')
                    print(str_model_name)
                    print(data_exec_x_copy)
                  if verbose_graph:
                    data_exec_x_copy.plot.kde(by=data_exec_x_copy.columns, alpha=0.5,title=str_model_name)

                  result_pred = model.predict(data_exec_x_copy) #prediction

                  if exec_performance == exec_accuracy:
                    result_accuracy = accuracy_score(y_true = original_outputs, y_pred = result_pred, normalize=normalize_performance) #accuracy
                  else:
                    if exec_performance == exec_auc:
                      result_accuracy = auc_score(original_outputs, result_pred) #AUC

                  result_pred = (result_pred == original_outputs) #binarization
                  list_clf_loop.append(str_model_name)
                  df_loop_of_models[str_model_name] = result_pred.astype(int)[:]
                  #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_model_name,'Acuracia':result_accuracy},ignore_index=True, verify_integrity=True)
                  df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_model_name, result_accuracy]
                else:
                  pass
  else:
    if exec_proposal == proposal_1_pool:
      
      model_copy = copy.deepcopy(model)

      if include_original_clf_data_pool:
        original_outputs = model_copy.predict(X_test)
        
        if exec_performance == exec_accuracy:
          result_accuracy = accuracy_score(y_true = original_outputs, y_pred = original_outputs, normalize=normalize_performance) #accuracy
        else:
          if exec_performance == exec_auc:
            result_accuracy = auc_score(original_outputs, original_outputs) #AUC
        
        result_pred = (original_outputs == original_outputs) #binarization
        

        str_model_name = 'Original model'
        list_clf_pool.append(str_model_name)
        df_loop_of_models[str_model_name] = result_pred.astype(int)[:]
        #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_model_name,'Acuracia':result_accuracy},ignore_index=True)
        df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_model_name, result_accuracy]
        if verbose:
          print('')
          print(str_model_name)
          print(X_test)
      
      ps = powerSetLimited(data_exec_x.columns, number_of_features_deletion) #powerset of all features
      
      for _, ps_c in enumerate(ps):
        data_train_x_copy = X_train.copy()
        data_test_x_copy = X_test.copy()
        for _, c in enumerate(ps_c):
          flag_none = True
          #remove feature c
          data_train_x_copy = data_train_x_copy.drop(c, axis='columns')
          data_test_x_copy = data_test_x_copy.drop(c, axis='columns')

        if len(data_train_x_copy.columns) == 0:
          break

        #train new model
        model_copy.fit(data_train_x_copy, y_train)

        result_pred = model_copy.predict(data_test_x_copy)
        if exec_performance == exec_accuracy:
          result_accuracy = accuracy_score(y_true = original_outputs, y_pred = result_pred, normalize=normalize_performance) #accuracy
        else:
          if exec_performance == exec_auc:
            result_accuracy = auc_score(original_outputs, result_pred) #AUC

        result_pred = (result_pred == original_outputs) #binarization
          
        str_model_name = 'Clf feature elimination: '
        for _, i in enumerate(ps_c):
          str_model_name = str_model_name + '"'+str(i)+'" '
        list_clf_pool.append(str_model_name)
        df_loop_of_models[str_model_name] = result_pred.astype(int)[:]
        #myfix#df_loop_of_models_performance = df_loop_of_models_performance.append({'Metodo':str_model_name,'Acuracia':result_accuracy}, ignore_index=True)
        df_loop_of_models_performance.loc[len(df_loop_of_models_performance)] = [str_model_name, result_accuracy]
        if verbose:
          print('')
          print(str_model_name)
          print(data_test_x_copy)

  #add V in index
  df_loop_of_models = df_loop_of_models.set_index('V' + str(i) for i in df_loop_of_models.index.values)

  if verbose:
    print('Resume Loop of model')
    print(df_loop_of_models.transpose())
    print()
    print('Resume Loop of model accuracy')
    print(df_loop_of_models_performance)

  df = df_loop_of_models.transpose()
  df.to_csv(path_content+path_irt+"/tabela_base_para_executar_irt.csv")
  if download_files:
    files.download(path_content+path_irt+"/tabela_base_para_executar_irt.csv") 

  df = df_loop_of_models_performance
  df.to_csv(path_content+path_irt+"/tabela_base_para_executar_irt_accuracy.csv",index=False)
  if download_files:
    files.download(path_content+path_irt+"/tabela_base_para_executar_irt_accuracy.csv")

  return  df_loop_of_models, df_loop_of_models_performance

def run_irt(datasetName):

  global path_dataset

  %rm -rf $path_content$path_irt'/irt_item_param.csv'
  %rm -rf $path_content$path_irt'/irt_item_param_new.csv'
  %rm -rf $path_content$path_irt'/OutExecution/theta_list.csv'
  %rm -rf $path_content$path_irt'/OutExecution/score_total.csv'

  !python decodIRT_MLtIRT.py -dir $path_irt -respMatrix $path_content$path_irt'/tabela_base_para_executar_irt.csv'

  url = path_content+path_irt+'/irt_item_param.csv'
  result_irt = pd.read_csv(url)
  if download_files:
    files.download(url) 
  
  result_irt_new = result_irt.copy()

  #save parameters of item by datset
  result_irt_dataset = result_irt.copy()
  result_irt_dataset.to_csv(path_content+path_irt+path_dataset+'/irt_item_param_'+datasetName+'.csv',index=False)
  if download_files:
    files.download(path_content+path_irt+path_dataset+'/irt_item_param_'+datasetName+'.csv')

  if irt_divine == False:
    result_irt_new['Adivinhacao'] = [0]*len(result_irt_new['Adivinhacao']) #anula os valores de adivinhação
  if irt_difficulty == False:
    result_irt_new['Dificuldade'] = [0]*len(result_irt_new['Dificuldade']) #anula os valores de dificuldade
  if irt_discriminate == False:
    result_irt_new['Discriminacao'] = [0]*len(result_irt_new['Discriminacao']) #anula os valores de Discriminacao

  result_irt_new.to_csv(path_content+path_irt+'/irt_item_param_new.csv',index=False)

  !python decodIRT_analysis.py -dir $path_irt -nameData OutExecution -respMatrix $path_content$path_irt'/tabela_base_para_executar_irt.csv' -IRTparam $path_content$path_irt'/irt_item_param_new.csv' -accur $path_content$path_irt'/tabela_base_para_executar_irt_accuracy.csv' -scoreAll -save $irt_method -missing

  
  %cat IRT_param_freq.txt
  %cp IRT_param_freq.txt $path_content$path_irt$path_dataset'/IRT_param_freq_'$datasetName'.txt'
  return result_irt_new, result_irt
  
def run_calc(name_of_features_x,datasetName):
  global path_dataset

  if exec_base_irt_score == exec_theta:
    url = path_content+path_irt+'/OutExecution/theta_list.csv'
    name_col = 'Theta'
  else:
    if exec_base_irt_score == exec_trueScore:
      url = path_content+path_irt+'/OutExecution/score_total.csv'
      name_col = 'Score'
  
  rank_theta = pd.read_csv(url)


  if download_files:
    files.download(url)
  rank_theta = rank_theta.sort_values(name_col,ascending=True)
  
  # if exec_proposal == proposal_1_pool:
  #   k = number_of_features*number_of_features_deletion*4
  # else:
  #   if exec_proposal == proposal_2_loop:
  #     k = number_of_features*len(exec_variation_method)*number_of_features_variation
  
  
  #rank_theta.plot.barh(x='Clf',y=name_col,figsize = (15,k),color='green')
  rank_theta = rank_theta.set_index(keys='Clf')

  if exec_proposal == proposal_2_loop:
    rank_theta_loop = rank_theta.sort_values(by=name_col, ascending=True)

    df_rank_final = pd.DataFrame(index=name_of_features_x, columns=[str('Final '+name_col)])
    
    for _, feature in enumerate(name_of_features_x):
      if exec_calc_theta == theta_sum:
        df_rank_final.loc[feature,str('Final '+name_col)] = sum(rank_theta_loop.filter(like='"'+feature+'"', axis='index')[name_col])
      else:
        if exec_calc_theta == theta_min:
          df_rank_final.loc[feature,str('Final '+name_col)] = min(rank_theta_loop.filter(like='"'+feature+'"', axis='index')[name_col])
        else:
          if exec_calc_theta == theta_mean:
            df_rank_final.loc[feature,str('Final '+name_col)] = statistics.mean(rank_theta_loop.filter(like='"'+feature+'"', axis='index')[name_col])

    df_rank_final = df_rank_final.sort_values(by=str('Final '+name_col), ascending=True)
    df_rank_final
  else:
    if exec_proposal == proposal_1_pool:

      rank_theta_pool = rank_theta.sort_values(by=name_col, ascending=True)

      df_rank_final = pd.DataFrame(index=name_of_features_x, columns=[str('Final '+name_col)])
      
      for _, feature in enumerate(name_of_features_x):
        if exec_calc_theta == theta_sum:
          df_rank_final.loc[feature,str('Final '+name_col)] = sum(rank_theta_pool.filter(like=feature, axis='index')[name_col])
        else:
          if exec_calc_theta == theta_min:
            df_rank_final.loc[feature,str('Final '+name_col)] = min(rank_theta_pool.filter(like=feature, axis='index')[name_col])
          else:
            if exec_calc_theta == theta_mean:
              df_rank_final.loc[feature,str('Final '+name_col)] = statistics.mean(rank_theta_pool.filter(like=feature, axis='index')[name_col])

      df_rank_final = df_rank_final.sort_values(by=str('Final '+name_col), ascending=True)


  
  df_rank_final.to_csv(path_content+path_irt+path_dataset+'/rank_final_'+datasetName+'.csv',index=True)
  return df_rank_final

def explainRankByEXirt(model, X_train, X_test, y_train, y_test,datasetName):
  global path_dataset

  path_dataset = '/'+datasetName
  dirByDataset(path_dataset)

  if(exec_in == exec_test):
    data_exec_x = X_test
    data_exec_y = y_test 
  else:
    data_exec_x = X_train
    data_exec_y = y_train

  
  N = 500
  if len(data_exec_y) > N:
    data_sample = data_exec_x
    data_sample['class'] = data_exec_y

    #stratifier sampler
    data_sample = data_sample.groupby('class', group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(data_sample))))).sample(frac=1).reset_index(drop=True)
    
    data_exec_y = data_sample['class']
    data_sample = data_sample.drop(labels='class', axis=1)
    data_exec_x = data_sample

  a, b = run_prepare(model, data_exec_x, data_exec_y, X_train, X_test, y_train, y_test)
  
  rirt_new, rirt = run_irt(datasetName)
  
  rank = run_calc(X_train.columns,datasetName)
  
  return list(rank.index), rank

## Load data and Pre-process

In [4]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(len(df_norm[column].unique()) > 1): #fix NaN generation
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        else:
          df_norm[column] = 0
    return df_norm

In [ ]:
#Select dataset name by openml link https://www.openml.org/search?sort=date

In [5]:
#load dataset by OpenML

dataset_name = "pc1"
dataset = openml.datasets.get_dataset(dataset_name)
X, Y, categorical_indicator, attribute_names = dataset.get_data(
                  dataset_format="dataframe", target=dataset.default_target_attribute)

#Preprocess Y and X numerics

if (Y.dtype != 'numeric'):
  Y = Y.astype(int)

for i,c in enumerate(X.columns):
  if (X[c].dtype != 'float64'):
    X = X.astype(float)

#Normalization
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) # 70% training and 30% test



## Creation and prediction model

In [6]:
model = RandomForestClassifier(200)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

## Global explanation rank

In [11]:
global_explanation_attributes, global_explanation_attributes_scores = explainRankByEXirt(model, X_train, X_test, y_train, y_test,dataset_name+'_rf')

mkdir: cannot create directory ‘/content/out_fig/pc1_rf’: File exists
mkdir: cannot create directory ‘/content/out_csv/pc1_rf’: File exists
mkdir: cannot create directory ‘/content/out_model/pc1_rf’: File exists
mkdir: cannot create directory ‘/content/out_irt/pc1_rf’: File exists


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

As frequencias dos parametros de item foram salvas \o/

/usr/local/lib/python3.8/dist-packages/scipy/optimize/optimize.py:2046: RuntimeWarning: invalid value encountered in double_scalars
  p = (xf - fulc) * q - (xf - nfc) * r
/usr/local/lib/python3.8/dist-packages/scipy/optimize/optimize.py:2047: RuntimeWarning: invalid value encountered in double_scalars
  q = 2.0 * (q - r)
Todos os valores de Theta foram salvos \o/
No handles with labels found to put in legend.

Os scores dos classificadores para todos os datasets foram salvos \o/

Porcentagem de itens com valores altos do parametro Discriminacao
Dataset 				 Percentual de itens
OutExecution                                    98%
------------------------------------------------------------
Porcentagem de itens com valores altos do parametro Dificuldade
Dataset 				 Percentual de itens
OutExecution                                     1%
------------------------------------------------------------
Porcentagem de itens com valores alto

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
global_explanation_attributes

['lOComment',
 'lOBlank',
 'uniq_Op',
 'I',
 'uniq_Opnd',
 'locCodeAndComment',
 'N',
 'lOCode',
 'V',
 'T',
 'total_Op',
 'total_Opnd',
 'E',
 'B',
 'v(g)',
 'loc',
 'iv(G)',
 'ev(g)',
 'D',
 'L',
 'branchCount']